## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jul. 29th Aug. 5th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [7]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,2024-08-02,Condo/Co-op,470 W Park Dr #104,Miami,FL,33172.0,260000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/470-W-Park-Dr-...,MARMLS,A11621874,N,Y,25.772929,-80.366603
117,PAST SALE,2024-08-02,Condo/Co-op,8821 W Flagler St #307,Miami,FL,33174.0,305000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8821-W-Flagler...,MARMLS,A11504428,N,Y,25.770484,-80.340351
39,PAST SALE,2024-08-02,Condo/Co-op,19751 SW 114th Ave #249,Miami,FL,33157.0,155000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19751-SW-114th...,MARMLS,A11583049,N,Y,25.582241,-80.376041
139,PAST SALE,2024-08-02,Condo/Co-op,17610 NW 73rd Ave Unit 206-5,Hialeah,FL,33015.0,265000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/17610-NW-73r...,MARMLS,A11515636,N,Y,25.934707,-80.320358
135,PAST SALE,2024-08-02,Condo/Co-op,1200 Euclid Ave #202,Miami Beach,FL,33139.0,280000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1200-Euc...,MARMLS,A11538318,N,Y,25.783085,-80.135553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,PAST SALE,2024-07-29,Condo/Co-op,1662 SE 27th Dr #106,Homestead,FL,33035.0,225000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/1662-SE-27...,MARMLS,A11568629,N,Y,25.444507,-80.453481
61,PAST SALE,2024-07-29,Condo/Co-op,55 SW 9th St #1806,Miami,FL,33130.0,810000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/55-SW-9th-St-3...,MARMLS,A11585673,N,Y,25.765661,-80.194394
133,PAST SALE,2024-07-29,Condo/Co-op,6423 Collins Ave #1408,Miami Beach,FL,33141.0,650000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6423-Col...,MARMLS,A11536015,N,Y,25.846616,-80.119699
110,PAST SALE,2024-07-29,Condo/Co-op,1151 N Liberty Ave Unit 1151F,Homestead,FL,33034.0,175000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/1151-N-Lib...,MARMLS,A11565344,N,Y,25.459069,-80.465348


In [8]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [9]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

156


In [10]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$673,346


In [11]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$510


In [12]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$105,041,900


In [14]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [13]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
131
Input Previous Week Condo Average Sales Price:
668297
Input Previous Week Condo Average PSF:
509
Input Previous Week Condo Sales total (ex: 198_000_000)
87500000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [14]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [15]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [16]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [17]:
pd.set_option('display.max_columns',None)

In [18]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
152,PAST SALE,2024-07-31,Condo/Co-op,18975 Collins Ave #5201,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,5.5,18975 COLLINS CONDO,3050.0,NaN,2020.0,NaN,1557.0,3850.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11484634,N,Y,25.95174,-80.119755,1,orange


In [19]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [20]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [21]:
m.save('index.html')

## Data snagger

In [22]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [23]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
152,PAST SALE,2024-07-31,Condo/Co-op,18975 Collins Ave #5201,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,5.5,18975 COLLINS CONDO,3050.0,NaN,2020.0,NaN,1557.0,3850.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11484634,N,Y,25.95174,-80.119755,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [24]:
df_top_ten['PRICE'].iloc[0]

4750000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [32]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_08052024


# CREATE TEMPLATE 

In [25]:
muni_set = set(df_top_ten['CITY'])

In [26]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [27]:
df_top_ten.reset_index(inplace=True,drop=True)

In [28]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [29]:
top_sale

'18975 COLLINS CONDO at 18975 Collins Ave #5201 in Sunny Isles Beach'

In [30]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [34]:
# df_top_ten

## RESUME HERE

In [33]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 18975 COLLINS CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,835,000M to $4,750,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Bay Harbor Islands, Miami Beach, Miami, Aventura, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 156 condo sales totaling $105,041,900 million from Jul. 29th to Aug. 5th. The previous week, brokers closed 131 condo sales totaling $87,500,000.

Last week’s units sold for an average of $673,346, higher than the $668,297 sales average from the previous week. 

In [34]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [35]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [36]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: 18975 COLLINS CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,835,000M to $4,750,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Bay Harbor Islands, Miami Beach, Miami, Aventura, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 156 condo sales totaling $105,041,900 million from Jul. 29th to Aug. 5th. The previous week, brokers closed 131 condo sales totaling $87,500,000.

Last week’s units sold for an average of $673,346, higher than the $668,297 sales average from the previous week. 

In [50]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Sunny-Isles-Beach/18975-Collins-Ave-33160/unit-5201/home/171996033


In [49]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Aventura/6000-Island-Blvd-33160/unit-2101/home/43342587


In [47]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/888-Brickell-Key-Dr-33131/unit-1503/home/42771809


In [48]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Bay-Harbor-Islands/10201-E-Bay-Harbor-Dr-33154/unit-404/home/145576667


In [41]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Surfside/8701-Collins-Ave-33154/unit-404/home/167094139


In [42]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/888-Brickell-Key-Dr-33131/unit-1503/home/42771809


In [43]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-07-31,Condo/Co-op,18975 Collins Ave #5201,Sunny Isles Beach,FL,33160.0,4750000.0,3.0,5.5,18975 COLLINS CONDO,3050.0,NaN,2020.0,NaN,1557.0,3850.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11484634,N,Y,25.951740,-80.119755,1,orange,18975 COLLINS CONDO at 18975 Collins Ave #5201...
1,PAST SALE,2024-08-01,Condo/Co-op,6000 Island Blvd #2101,Aventura,FL,33160.0,4010000.0,5.0,5.5,BELLAMARE @ WILLIAMS ISLA,4109.0,NaN,2005.0,NaN,976.0,4088.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11588149,N,Y,25.942761,-80.135433,2,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
2,PAST SALE,2024-07-29,Condo/Co-op,1000 S Pointe Dr #1404,Miami Beach,FL,33139.0,3700000.0,3.0,3.0,MURANO AT PORTOFINO CONDO,2190.0,NaN,2002.0,NaN,1689.0,3024.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11604509,N,Y,25.768527,-80.137591,3,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
3,PAST SALE,2024-08-02,Condo/Co-op,2127 Brickell Ave #1103,Miami,FL,33129.0,3510000.0,3.0,3.5,BRISTOL TOWER CONDO,2700.0,NaN,1993.0,NaN,1300.0,3418.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11533034,N,Y,25.752160,-80.199342,4,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #1103...
4,PAST SALE,2024-07-29,Condo/Co-op,5500 Island Estates Dr #1107,Aventura,FL,33160.0,3425000.0,3.0,4.5,PRIVE CONDO,2877.0,NaN,2017.0,NaN,1190.0,7482.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/5500-Island...,MARMLS,A11574723,N,Y,25.946653,-80.130529,5,blue,PRIVE CONDO at 5500 Island Estates Dr #1107 in...
5,PAST SALE,2024-07-30,Condo/Co-op,8701 Collins Ave #404,Miami Beach,FL,33154.0,3225000.0,2.0,2.5,8701 COLLINS AVE CONDO,1594.0,NaN,2019.0,NaN,2023.0,10486.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11296790,N,Y,25.872197,-80.121210,6,blue,8701 COLLINS AVE CONDO at 8701 Collins Ave #40...
6,PAST SALE,2024-07-31,Condo/Co-op,15701 Collins Ave #3803,Sunny Isles Beach,FL,33160.0,3050000.0,2.0,2.5,15701 COLLINS CONDO,1605.0,NaN,2020.0,NaN,1900.0,2528.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11531661,N,Y,25.921944,-80.121716,7,blue,15701 COLLINS CONDO at 15701 Collins Ave #3803...
7,PAST SALE,2024-07-29,Condo/Co-op,901 Brickell Key Blvd #2508,Miami,FL,33131.0,2950000.0,4.0,4.5,CARBONELL CONDO,2860.0,NaN,2005.0,NaN,1031.0,2600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11404549,N,Y,25.768991,-80.186892,8,blue,CARBONELL CONDO at 901 Brickell Key Blvd #2508...
8,PAST SALE,2024-08-01,Condo/Co-op,10201 E Bay Harbor Dr #404,Bay Harbor Islands,FL,33154.0,1950000.0,3.0,3.0,SERENO RESIDENCES CONDO,1501.0,NaN,2016.0,NaN,1299.0,2354.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11542692,N,Y,25.892681,-80.130898,9,blue,SERENO RESIDENCES CONDO at 10201 E Bay Harbor ...
9,PAST SALE,2024-07-30,Condo/Co-op,888 Brickell Key Dr #1503,Miami,FL,33131.0,1835000.0,3.0,3.0,ONE TEQUESTA POINT CONDO,2050.0,NaN,1995.0,NaN,895.0,1799.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/888-Brickell-K...,MARMLS,A11589589,N,Y,25.769244,-80.184873,10,blue,ONE TEQUESTA POINT CONDO at 888 Brickell Key D...


## Time on Market Calculator

In [46]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 5, 22) ## List (Earlier) date
date2 = datetime(2024, 8, 1) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

71
